<div class = "alert alert-warning" align="center"> 
    <img src="https://www.egd.mg/wp-content/uploads/2020/01/cropped-logo-home-110px.png" align="left" width="290"  margin="50px"> 
    <center><h1>Lycée français de Tananarive</h1> 
        <h2> Jupyter Hub,  Spécialité NSI </h2></center>
</div>

# Traitement de données en table
## 1. Position du problème : Le code Postal
<img src="laposte.png" style="float: right; margin-left: 10px;"  height="50%" width="50%"/>
Sur le site de la poste un formulaire permet de trouver le code postal de n'importe quelle commune. Nous allons à partir du fichier CSV utilisé par ce site d'essayer d'en faire de même avec un programme écrit en Python qui pourra nous donner différentes informations quant aux communes françaises.

le travail à réaliser se fera en plusieurs étapes :
- Etudier et nettoyer le fichier csv `code.csv`
- Charger ce fichier en tant que table de données
- Opérer différentes opérations sur cette table.
- Positionner sur une carte les différentes communes d'un département.

## 2 Etude et nettoyage du fichier CSV.
### 2.1 Etude du fichier CSV.
Ouvrir le fichier CSV et répondre dans la cellule ci-dessous aux questions suivantes :
1. Quels sont les attributs des éléments de ce fichiers ?
2. Combien ce fichier comporte-t-il d'éléments ?
3. Quels sont les éléments de type `float` que contient ce fichier ?
4. Comment s'appelle l'attribut dont dont les valeurs sont vides pour tous les enregistrements de la table ?
5. Quel séparateur est utilisé dans cette table ?

YOUR ANSWER HERE

### 2.2 Création de la table de données 
**En vous appuyant sur vos connaissances de cours créez une table de données nommée `commune`composée de dictionnaires pour chacun des enregistrements, où chaque clef représente un attribut de la table**

Dans un premier temps on se contentera de créer la table, sans se précocuper du typage des attributs.

In [25]:
import csv
fichier = open('codes.csv')
commune = list(csv.DictReader(fichier, delimiter = ";"))

In [26]:
assert commune[14]['Code_commune_INSEE'] == '91137'
assert commune[5214]['Nom_commune'] == 'ANTIBES'
assert commune[8762]['Code_postal'] == '32130'
assert commune[7540]['Ligne_5'] == ''
assert commune[25]['coordonnees_gps'] == '48.368305642, 2.07601234971'

### 2.3 Validation de la table.
**Chaque élément de la table contient un attribut nommé `Ligne_5` qui est vide pour tous les enregistrements, écrire un script duplique la table `commune` en créant une nouvelle table `commune_valide` identique à la première mais en supprimant cette clef inutile dans tous les enregistrements.**

On pourra éventuellement utiliser la commande qui permet de supprimer une clef dans un dictionnaire est `del dict[clef]`, mais ce n'est pas obligatoire.

In [27]:
import csv
fichier = open('commune_valide.csv')
commune = list(csv.DictReader(fichier, delimiter = ";"))
for i in commune:
    del i['Ligne_5']
commune_valide = commune

In [28]:
assert commune_valide[14]['Code_commune_INSEE'] == '91137'
assert commune_valide[5214]['Nom_commune'] == 'ANTIBES'
assert commune_valide[8762]['Code_postal'] == '32130'
assert len(commune_valide[18]) == 5
assert len(commune_valide) == 39201

## 3. Recherche dans la table.
### 3.1 Interrogation de la table.
**Ecrire un fonction `code_postal(nom_commune)` qui renvoie le code postal, sous forme d'une chaine de caractéres, d'une commune dont le nom est passé en paramètre. La fonction doit retournée `None` si le nom de la commune n'est pas dans la liste**

In [29]:
def code_postal(nom_commune):
    return next((i['Code_postal'] for i in csv.DictReader(open('commune_valide.csv'), delimiter = ";") if i['Nom_commune'] == nom_commune), None)

In [30]:
assert code_postal('TOULOUSE') == '31000'
assert code_postal('MURET') == '31600'
assert code_postal('LE TAMPON') == '97418'
assert code_postal('TANANARIVE') == None

### 3.2 recherche d'éléments
**Ecrire une fonction `ville(code_postal)` qui renvoie une liste dont les éléments sont le nom des communes qui ont comme attribut le code postal passé en paramètre**

In [31]:
def ville(code_postal):
    return [i['Nom_commune'] for i in list(csv.DictReader(open('commune_valide.csv'), delimiter = ";")) if i['Code_postal'] == code_postal]

In [32]:
assert ville('24000') == ['PERIGUEUX']
assert ville('31430') == ['MONTOUSSIN','CASTIES LABRANDE','LUSSAN ADEILHAC','SENARENS','MARIGNAC LASCLARES','MONTEGUT BOURJAC','POUY DE TOUGES','ST ARAILLE','LE FOUSSERET','POLASTRON','ST ELIX LE CHATEAU','CASTELNAU PICAMPEAU','FUSTIGNAC','GRATENS']
assert ville('75001') == ['PARIS 01']

### 3.3 Projection de tables.
**Ecrire un fonction qui renvoie une version simplifiée de la table `commune_valide`. Les éléments de cette table ne doivent contenir que deux attributs : `Nom_commune` et `Code_postal`**.


In [35]:
def table_simplifiee(table):
    return [{"Nom_commune": d["Nom_commune"], "Code_postal": d["Code_postal"]} for d in table]

In [36]:
assert table_simplifiee(commune_valide)[2]['Nom_commune'] == 'ST GERMAIN LE CHATELET'
assert table_simplifiee(commune_valide)[1475]['Code_postal'] == '89340'
assert len(table_simplifiee(commune_valide)[4475]) == 2
assert len(table_simplifiee(commune_valide)) == 39201

### 3.4 Agrégation de tables
En France les deux premiers chiffres du `Code_commune_INSEE` correspondent au numéro du département. Par exemple, 'GRATENS' a pour code commune INSEE 31430, cette commune se trouve donc dans le département 31, la haute garonne.

**Ecrire une fonction `nbre_commune(dpt,table)`, qui renvoie le nombre de commune dans le département passé en paramètre**

- Données : 
    - `dpt` : Chaîne de caractères composée de 2 caractères
    - `table` Liste de dictionnaires (table `commune_valide` par exemple)
- Retourne :
    - Un entier qui reprèsente le nombre de commune dans le département.

In [49]:
def nbre_commune(dpt, table):
    return sum(1 for i in table if i['Code_commune_INSEE'][:2] == dpt)

In [50]:
assert nbre_commune('31',commune_valide) == 600
assert nbre_commune('2A',commune_valide) == 136
assert nbre_commune('97',commune_valide) == 214